In [2]:
%run runaway_functionsv3

In [3]:
cl = Cluster("Berkeley_97")
run = cl.runaways()
run
# ra = cl.runaways_all()
# thiso = cl.theoretical_isochrone()
# ra['Temp. Est'] = ra["Teff"]



3.1 7.1 -0.1


RA_ICRS_1,DE_ICRS_1,rgeo,Teff,Temp. Est,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Gmag,BP-RP,BPmag,RPmag,b_rgeo,B_rgeo,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord,rmRA,rmDE,v_pec,logg,RV,e_RV,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6
deg,deg,pc,K,K,mas,mas,,,,,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,mag,mag,mag,mag,pc,pc,,,,,"deg,deg,pc",mas / yr,mas / yr,km / s,log(cm.s**-2),km / s,km / s,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,int32,str11,int64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,SkyCoord,float64,float64,float64,float64,float64,float32,float64,float32,float32,float32,float32,float32,str1,str1
339.89581711423,58.99547917199,2620.36230000,9168.0,15330.0,0.0202,0.0197,1.5205,--,--,2008384504995669248,0.3560,0.0241,-0.379,-1.352,0.025,0.024,1.086,15.384143,1.182070,15.883185,14.701116,2461.53345000,2759.61401000,0.00277,0.003852,0.004229,0.008081,"339.89581711423,58.99547917199,2620.3623",2.318000026702881,0.47699999618530264,29.394022464312897,3.7061,--,--,13221.76503,3.482,6055,14.81,1.044e+04,18.25,--,--


In [90]:
cl.changeParam(('Av',3.1))
cl.changeParam(('logage',7.1))

Changed Av         2.54 --> 3.10
Changed logage     7.59 --> 7.10


In [4]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion,PixCoord, PointSkyRegion, PointPixelRegion, LineSkyRegion, LinePixelRegion
%matplotlib qt
allrun = cl.runaways()[:]
allrun_coord_now = SkyCoord(ra=allrun['RA_ICRS_1'], 
                        dec=allrun['DE_ICRS_1'],
                        distance=allrun['rgeo'], 
                        pm_ra_cosdec=allrun['rmRA'],
                        pm_dec=allrun['rmDE'],
                        obstime=Time('J2000')+500*u.kyr)

allrun_coord_earlier = allrun_coord_now.apply_space_motion(dt=-100*u.kyr)


with fits.open(f'./Clusters/{cl.name}/{cl.name}_extra10pc.fits') as fits_file:
    image = fits_file[0]
    wcs = WCS(image.header)
    fig, ax = plt.subplots(subplot_kw={'projection': wcs}, figsize=(8, 8))
    ax.imshow(image.data, 
              cmap='gray')
    ax.set_xlabel('Right Ascension (hms)')
    ax.set_ylabel('Declination (degrees)')

    c = SkyCoord("98.5582 +8.0381", distance=1456, unit=(u.deg ,u.deg, u.pc))
    radius = Angle(0.3, 'deg')
    region = CircleSkyRegion(c,radius)
    region_pix = region.to_pixel(wcs)
    region_pix.plot(ax=ax, 
                    color='red', 
                    lw=2)
    
    start_skys = allrun_coord_now
    end_skys = allrun_coord_earlier

    region_sky = [LineSkyRegion(start=start_sky, end=end_sky) for start_sky,end_sky in zip(allrun_coord_now,allrun_coord_earlier)]
    region_sky_pix = [region_sky_.to_pixel(wcs) for region_sky_ in region_sky]
    for c in allrun_coord_now:
        point_sky = PointSkyRegion(center=c)
        point_pix = point_sky.to_pixel(wcs)
        point_pix.plot(ax=ax,
                       color='yellow')
    for region_pix in region_sky_pix:
        region_pix.plot(ax=ax,
                        color='cyan')

3.1 7.1 -0.1


/usr/lib/python3/dist-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [37]:
dias2021 = Table.read("dias2021.tsv", format="ascii.ecsv")
maskplx = dias2021['Plx'] > 0.3
maskage = dias2021['logage'] < 7.7
workclusters = []
for clustername in dias2021[maskplx & maskage][:]['Cluster']:
    if clustername not in ['ASCC_79','BH_164','BH_23','Collinder_135','Collinder_140','Gulliver_9','IC_2391','IC_2602','Mamajek_1','Platais_8','UPK_535','UPK_606','UPK_640','Berkeley_59','COIN-Gaia_37','Ivanov_4','LP_1937','Sigma_Ori','UBC_632']:
        workclusters.append(clustername)
        # print(clustername)
        # cl = Cluster(clustername)
        # cl.prepare_trace()
# cl.stars_in_region()
# cl.stars_in_region()

In [ ]:
workclusters_mask = [cluster in workclusters for cluster in dias2021['Cluster']]
workclusterstable = dias2021[workclusters_mask]

rvexists = workclusterstable['RV'].mask
nrvgood = workclusterstable['NRV'] > 4

workclusterstable[~rvexists & nrvgood]

In [ ]:
def runaways_from(clustername):
    cl = Cluster(clustername)
    fs4giesler = cl.fast_stars_in_region()
    outputs = os.listdir(f"/home/surodeep/suro_aiu/traceback/cluster_runaway/{cl.name}/runaways/")
    linenos = []
    for output in outputs:
        #print(output)
        if 'run' in output:
            linenos.append(int(output.split("+")[1].replace(".out","")))
    linenos.sort()
    # print(linenos)
    i=np.array(linenos)-3
    def source_of(lineno, input_table):
        return input_table[lineno-2]['Source']
    return fs4giesler[i]
for cluster in workclusters:
    t = runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Teff']
    
    if t>10000:
        print(cluster, t,runaways_from(cluster)[runaways_from(cluster)['Teff'].argmax()]['Source'])